In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

host="rhea.isegi.unl.pt"
port="27018"
user="mongo_classes_rhea"
password="ULWQFGfyZWedCMg5PHKZTYGFxmqSEZ6QsMYV"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [2]:
db = client.hw3_example

In [3]:
# Lets clean up the collections in this db
r = list(db.list_collections())

for collection in r:
    print(collection['name'])

collectionX_embedded
collection3_embeded
collection2
collection1
places


In [4]:
db['collection1'].drop()
db['collection2'].drop()
db['collection3'].drop()

### Important notice: There are several correct methods to upload data into MongoDB, here we only show one basic example.

### Important notice: We are only showing a direct import to mongoDB, there are some differences between sql and mongoDB that can be 'solved' in the notebook

# Import with pandas (csv)

### When csv is small enoght to import to pandas

In [5]:
import pandas as pd

In [6]:
path = 'hw3_extra_materials/csv/sample1.csv'

df_csv = pd.read_csv(path)

FileNotFoundError: [Errno 2] No such file or directory: 'hw3_extra_materials/csv/sample1.csv'

In [ ]:
df_csv.head(2)

In [ ]:
db.collection1.insert_many(df_csv.to_dict('records'))

In [ ]:
db.collection1.find_one()

### When csv is too large to be loaded by pandas

In [ ]:
path = 'hw3_extra_materials/csv/sample1.csv'

In [ ]:
def upload_to_db(df, collection):
    db[collection].insert_many(df.to_dict('records'))

In [ ]:
chunksize = 2 #number of rows per chunk

collection = 'collection1'

with pd.read_csv(path, chunksize=chunksize) as reader:
    for chunk in reader:
        upload_to_db(chunk, collection)
        print(db.collection1.count_documents({}))

# Import Json

In [ ]:
import json

## Example 1

In [ ]:
path = 'hw3_extra_materials/json/sample1.json'


with open(path) as json_file:
    json_2 = json.load(json_file)

In [ ]:
json_2

In [ ]:
data = [json_2] # Using insert many imlplies a list of documents

In [ ]:
r = db.collection2.insert_many(data)

In [ ]:
r.inserted_ids

## Example 2

In [ ]:
path = 'hw3_extra_materials/json/sample2.json'


with open(path) as json_file:
    json_4 = json.load(json_file)

In [ ]:
json_4 # There is a list here

In [ ]:
data = json_4['people']

In [ ]:
r = db.collection2.insert_many(data)

In [ ]:
r.inserted_ids

### You can also use Mongo Database Tools - mongoimport

Instalation Required: https://docs.mongodb.com/database-tools/

# We have data in MongoDB, but not a model!
## Lets create an embedded collection from collections 1 and 2

In [ ]:
r = list(db.collection1.find().limit(2))

r

In [ ]:
list(db.collection2.find({}))

## Collection1's "client_number" key represents the "number" key in collection2
### Let's create an embbeded collection with collection1 embbeded inside of collection2

In [ ]:
query_1 = {
        "$lookup":{
           "from": "collection1",
           "localField": "client_number",
           "foreignField": "number",
           "as": "purchases"
        }
    }




# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "collection3_embeded",
            "whenMatched": "merge"
        } 
    }

#query_2 = { '$out' : "collection3_embeded" }






pipeline = [query_1, query_2]

r = db.collection2.aggregate(pipeline)

list(r)

In [ ]:
r = list(db.collection3_embeded.find({}))

r[0]

# SQL Code, regarding the questions from HW3

## Disclaimer: This is an early draft in sql of the questions asked, some of them might not be fully complete

### Design Requirements

You have been informed that the WWI has the following query requirements to the database.

1. The web team needs:  
* From which states are our suppliers from?
```
SELECT StateProvinceName, COUNT(*)
FROM WWI.Application_Countries
INNER JOIN WWI.Application_StateProvinces ON Application_Countries.CountryID = Application_StateProvinces.CountryID
INNER JOIN WWI.Application_Cities AC on Application_StateProvinces.StateProvinceID = AC.StateProvinceID
INNER JOIN WWI.Purchasing_Suppliers ON AC.CityID = Purchasing_Suppliers.DeliveryCityID
GROUP BY StateProvinceName
ORDER BY 2 DESC
```
* From which state province are the customers who have a higher credit limit?
```
SELECT StateProvinceName, AVG(CreditLimit)
FROM WWI.Application_Countries
INNER JOIN WWI.Application_StateProvinces ON Application_Countries.CountryID = Application_StateProvinces.CountryID
INNER JOIN WWI.Application_Cities AC on Application_StateProvinces.StateProvinceID = AC.StateProvinceID
INNER JOIN WWI.Sales_Customers ON AC.CityID = Sales_Customers.DeliveryCityID
GROUP BY StateProvinceName
ORDER BY 2 DESC
```
2. The warehouse group needs:  
* To know which items get ordered together the most?
```
SELECT TOP 30 ol2.Description,ol1.Description, count(*)
FROM WWI.Sales_OrderLines ol1
INNER JOIN WWI.Sales_OrderLines ol2 on ol1.OrderID = ol2.OrderID
WHERE ol2.Description > ol1.Description
GROUP BY ol2.Description,ol1.Description
ORDER BY 3 DESC
```
* Which items get ordered the most in bulk (bigger amounts)?
```
SELECT TOP 3 StockItemName, COUNT(Quantity), AVG(Quantity)
FROM WWI.Sales_OrderLines
INNER JOIN WWI.Warehouse_StockItems ON Warehouse_StockItems.StockItemID=Sales_OrderLines.StockItemID
WHERE Quantity > 0
GROUP BY StockItemName
```
* Which customers have delivery addresses under 10km of distance? 
```
SELECT TOP 30 c1.PostalPostalCode, c2.PostalPostalCode,c1.PostalAddressLine2, c2.PostalAddressLine2, c1.DeliveryLocation.STDistance(c2.DeliveryLocation) / 1000
FROM WWI.Sales_Customers as c1
JOIN WWI.Sales_Customers c2 ON c1.CustomerID <> c2.CustomerID
WHERE c1.CustomerID > c2.CustomerID
   and c1.DeliveryLocation.STDistance(c2.DeliveryLocation) != 0
   and c1.DeliveryLocation.STDistance(c2.DeliveryLocation)/1000 < 10
ORDER BY 5
```
3. The CFO:  
* Would like to know the monthly order count?
```
SELECT YEAR(OrderDate), MONTH(OrderDate), COUNT(*)
FROM WWI.Sales_Orders
GROUP BY YEAR(OrderDate), MONTH(OrderDate)
ORDER BY YEAR(OrderDate), MONTH(OrderDate)
```
* Would like to know the average monthly sales prices?
```
SELECT YEAR(OrderDate), MONTH(OrderDate), AVG(Quantity * UnitPrice)
FROM WWI.Sales_Orders
INNER JOIN WWI.Sales_OrderLines ON Sales_Orders.OrderID = Sales_OrderLines.OrderID
GROUP BY YEAR(OrderDate), MONTH(OrderDate)
ORDER BY YEAR(OrderDate), MONTH(OrderDate)
```
* Would like to know the yearly expenditures with suppliers (per supplier name)?
```
SELECT YEAR(TransactionDate), SupplierName,  SUM(TransactionAmount)
FROM WWI.Purchasing_SupplierTransactions
INNER JOIN WWI.Purchasing_Suppliers ON Purchasing_SupplierTransactions.SupplierID = Purchasing_Suppliers.SupplierID
WHERE SupplierInvoiceNumber is not null
GROUP BY YEAR(TransactionDate), SupplierName
ORDER BY 1
```
4. Partnerships:
* Would like to know what's the most common payment type?
```
SELECT PaymentMethodName, count(*)
FROM WWI.Purchasing_SupplierTransactions
INNER JOIN WWI.Application_PaymentMethods ON Purchasing_SupplierTransactions.PaymentMethodID = Application_PaymentMethods.PaymentMethodID
INNER JOIN WWI.Sales_CustomerTransactions ON Sales_CustomerTransactions.PaymentMethodID = Application_PaymentMethods.PaymentMethodID
GROUP BY PaymentMethodName
```

* Which supplier of `Novelty Goods Supplier` as the most transactions?
```
SELECT SupplierCategoryName, SupplierName, count(*)
FROM WWI.Purchasing_Suppliers
JOIN WWI.Purchasing_SupplierCategories ON Purchasing_Suppliers.SupplierCategoryID = Purchasing_SupplierCategories.SupplierCategoryID
JOIN WWI.Purchasing_SupplierTransactions ON Purchasing_Suppliers.SupplierID = Purchasing_SupplierTransactions.SupplierID
GROUP BY  SupplierCategoryName,  SupplierName
ORDER BY 1,3 desc
```
5. The marketing team:  
* Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who's customers brought the most money)?
```
SELECT TOp 30 FullName, count(*), sum(Quantity*UnitPrice)
FROM WWI.Application_People
JOIN WWI.Sales_Invoices ON Application_People.PersonID = Sales_Invoices.SalespersonPersonID
JOIN WWI.Sales_InvoiceLines ON Sales_Invoices.InvoiceID = Sales_InvoiceLines.InvoiceID
WHERE year(InvoiceDate) = 2013
GROUP BY FullName
ORDER BY 2 DESC
```
---

# Indexes Expansion

In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

from pymongo import DESCENDING, ASCENDING, TEXT, GEO2D

host="rhea.isegi.unl.pt"
port="27018"
user="mongo_classes_rhea"
password="ULWQFGfyZWedCMg5PHKZTYGFxmqSEZ6QsMYV"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.beer

## Text index
### https://docs.mongodb.com/manual/text-search/

In [ ]:
db.reviews.find_one()

In [ ]:
for index in db.reviews.list_indexes():
    print(index['name'])

In [ ]:
# drop index

try:
    db.reviews.drop_index('text_00')
except Exception as e: print(e)

In [ ]:
# Index goes here

db.reviews.create_index(
    [('text', TEXT)], # We've imported the TEXT variable to explicitly define an index as text
    name='text_00', 
)

In [ ]:
list(db.reviews.find({'$text':{'$search': "great"}}).limit(10))

In [ ]:
# drop index after

db.reviews.drop_index('text_00')

### Geospatial Index

In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

host="rhea.isegi.unl.pt"
port="27018"
user="mongo_classes_rhea"
password="ULWQFGfyZWedCMg5PHKZTYGFxmqSEZ6QsMYV"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")


db = client.hw3_example

db['places'].drop()

In [ ]:
import numpy as np

path = 'hw3_extra_materials/csv/geo_data.csv'


geo_df = pd.read_csv(path)

In [ ]:
geo_df.head()

In [ ]:
geo_dict = []

for row in geo_df.values:
    record = {
        'nome': row[0],
        'coords': [row[1], row[2]]
    }
    
    geo_dict.append(record)

In [ ]:
# Create new db places with "geospatial" data [lat, lon]

result = db.places.insert_many(geo_dict)  

In [ ]:
# Geospatial Index Creation
db.places.create_index([("coords", GEO2D)])

In [ ]:
coords_1 = list(db.places.find({'nome':'Lisboa'}))[0]['coords']
coords_2 = list(db.places.find({'nome':'Setúbal'}))[0]['coords']

In [ ]:
query = {"coords": {"$near": coords_1}}
    
list(db.places.find(query, {'_id':0}).limit(10))

In [ ]:
from bson.son import SON

query = {"coords": SON([("$near", coords_1), ("$maxDistance", .1)])}

list(db.places.find(query, {'_id':0}).limit(10))

In [ ]:
query = {"coords": {"$within": {"$box": [coords_1, coords_2]}}}

list(db.places.find(query, {'_id':0}).limit(10))

In [ ]:
query = {"coords": {"$within": {"$center": [coords_1, 6]}}}

list(db.places.find(query, {'_id':0}).limit(10))